In [143]:
import numpy as np
import cv2
import mediapipe as mp
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_TESSELATION
import json
import networkx as nx
import matplotlib.pyplot as plt
import math
import scipy.sparse
import pathlib

In [144]:
mp_face_mes = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mes.FaceMesh(max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [145]:
# frame = cv2.imread('angry.jpeg') # read the image *** change the path to your image

In [146]:

def preprocess_frame(frame):
    H, W, _ = frame.shape
    rgb_image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results_mesh = face_mesh.process(rgb_image)
    mesh_points=np.array([np.multiply([p.x, p.y], [W, H]).astype(int) for p in results_mesh.multi_face_landmarks[0].landmark])
    nose_tip = mesh_points[4]
    forehead = mesh_points[151]
    scale_factor = np.linalg.norm(forehead - nose_tip)
    if np.isclose(scale_factor, 0):
        scale_factor = 1e-6
    return results_mesh, mesh_points, scale_factor

In [147]:
# results_mesh, mesh_points, scale_factor = preprocess_frame(frame)

In [148]:
def gera_grafos(results_mesh, mesh_points, scale_factor):
    graph = nx.Graph()
    vertices = []
    edges = []
    distances = []

    for edge in FACEMESH_TESSELATION:
        pointA_index = edge[0]
        pointB_index = edge[1]
        if [pointA_index, pointB_index] not in edges:
            edges.append([pointA_index, pointB_index])
            graph.add_edge(pointA_index, pointB_index)
            x_diff = results_mesh.multi_face_landmarks[0].landmark[pointA_index].x - results_mesh.multi_face_landmarks[0].landmark[pointB_index].x
            y_diff = results_mesh.multi_face_landmarks[0].landmark[pointA_index].y - results_mesh.multi_face_landmarks[0].landmark[pointB_index].y
            x_diff /= scale_factor
            y_diff /= scale_factor
            x_diff *= 100000
            y_diff *= 100000
            distances.append(math.sqrt(x_diff**2 + y_diff**2))
        if pointA_index not in vertices:
            vertices.append(pointA_index)
            graph.add_node(pointA_index)
        if pointB_index not in vertices:
            vertices.append(pointB_index)
            graph.add_node(pointB_index)
    return graph

In [149]:
def plot_graph(graph, mesh_points):
    plt.figure(figsize=(10,10))
    nx.draw_networkx(graph, pos=mesh_points, node_size=10, node_color='black', edge_color='black', with_labels=False)
    plt.show()

In [150]:
def get_matrix_adj(graph):
    matrix_adj = nx.adjacency_matrix(graph)
    sparce_matrix = scipy.sparse.csr_matrix(matrix_adj)
    return sparce_matrix    

In [151]:
def save_adjacency_matrix(adjacency_matrix, filename):
    path_name = pathlib.Path(filename).parent.absolute()
    print('path_name ', path_name)
    emotion = path_name.name.split('_')[1]
    print('emotion ',emotion)
    path_name = path_name.parent.absolute()
    print(path_name)
    path_name = path_name.joinpath(f'{emotion}_adj')
    print('save_path ',path_name)
    # separete the filename from the path
    filename = pathlib.Path(filename).absolute()
    print('filename ', filename)
    # create te folder if it doesn't exist
    path_name.mkdir(parents=True, exist_ok=True)
    # save the npz file in the path_name
    file_out = path_name.joinpath(f'{filename.name}.npz')
    print(file_out)

    
    scipy.sparse.save_npz(file_out, adjacency_matrix)

In [152]:
def pipeline(img, plot=False):
    name = img.split('.')[0]
    frame = cv2.imread(img)  # read the image *** change the path to your image
    results_mesh, mesh_points, scale_factor = preprocess_frame(frame)
    graph = gera_grafos(results_mesh, mesh_points, scale_factor)
    adjacency_matrix = get_matrix_adj(graph)

    save_adjacency_matrix(adjacency_matrix, name)
    if plot:
        plot_graph(graph, mesh_points)

In [153]:
# img_angry = 'face_angry/0.jpg'

In [154]:
# pipeline("img_angry", plot=True)

## Aplicando a pipeline para o dataset de treino

In [155]:
### começando pelas fotos do tipo angry

import pathlib

actual_path = pathlib.Path().absolute()

# subindo um nível

path = actual_path.parent

angry_path = path / 'face_angry'

# constando o número de arquivos jpg

import os

angry_files = os.listdir(angry_path)

print(len(angry_files))

5017


In [156]:
count = 0
for img in angry_files:
    if img.endswith('.jpg'):
    print(img)
    # img = cv2.imread(str(angry_path / img))
    path_img = angry_path / img
    print("path",path_img)
    pipeline(str(path_img), plot=False)
    if count == 10:
        break

0.jpg
path c:\Users\rodri\Documents\Rodrigo\Insper\SextoSemestre\Facial-Emotion-Classification-Graph_Fork\face_angry\0.jpg
path_name  c:\Users\rodri\Documents\Rodrigo\Insper\SextoSemestre\Facial-Emotion-Classification-Graph_Fork\face_angry
emotion  angry
c:\Users\rodri\Documents\Rodrigo\Insper\SextoSemestre\Facial-Emotion-Classification-Graph_Fork
save_path  c:\Users\rodri\Documents\Rodrigo\Insper\SextoSemestre\Facial-Emotion-Classification-Graph_Fork\angry_adj
filename  c:\Users\rodri\Documents\Rodrigo\Insper\SextoSemestre\Facial-Emotion-Classification-Graph_Fork\face_angry\0
c:\Users\rodri\Documents\Rodrigo\Insper\SextoSemestre\Facial-Emotion-Classification-Graph_Fork\angry_adj\0.npz
1.jpg
path c:\Users\rodri\Documents\Rodrigo\Insper\SextoSemestre\Facial-Emotion-Classification-Graph_Fork\face_angry\1.jpg
path_name  c:\Users\rodri\Documents\Rodrigo\Insper\SextoSemestre\Facial-Emotion-Classification-Graph_Fork\face_angry
emotion  angry
c:\Users\rodri\Documents\Rodrigo\Insper\SextoSemes

TypeError: 'NoneType' object is not subscriptable